**Testing the DSAPM**

In [ ]:
# Initialization
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import cv2
import os
from google.colab.patches import cv2_imshow
from tensorflow.keras.models import load_model
from math import log10, sqrt
from sklearn import preprocessing

# Load the DSAPM model
model = load_model("models/dsapm.h5", compile=False)

# Load the test data
all_images_test = []
im_dir = "/content/drive/MyDrive/Test_Data"
list_of_files = sorted(os.listdir(im_dir), reverse=False)

for im_folder in list_of_files:
    list_of_img_files = sorted(os.listdir(os.path.join(im_dir, im_folder)), reverse=False)
    for image_file in list_of_img_files:
        image_path = os.path.join(im_dir, im_folder, image_file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (128, 128))
        all_images_test.append([image])

# Ensure correct total images before reshaping
print(f"Total images loaded: {len(all_images_test)}")

# Reshape test data into (360, 20, 128, 128, 1)
all_images_test = np.array(all_images_test).reshape(360, 20, 128, 128, 1)

# Normalize the data
all_images_test = all_images_test / 255.0

# Compute MSE
MSE = []
for j in range(360):
    example = all_images_test[j, :, :, :, :]
    frames = example[1:20, :, :, :, :]  # Input frames
    original_frames = example[:, :, :, :]  # Ground truth

    new_prediction = model.predict(np.expand_dims(frames, axis=0))
    new_prediction = np.squeeze(new_prediction, axis=0)
    predicted_frame = np.expand_dims(new_prediction[-1, ...], axis=0)

    frames = np.concatenate((frames, predicted_frame), axis=0)
    new_frames = frames[-1, :, :, :, :]

    mse = np.square(np.subtract(new_frames, original_frames[1, :, :, :, :])).mean()
    MSE.append(mse)

# Compute PSNR
max_pixel = 1.0
PSNR = [20 * log10(max_pixel / sqrt(mse)) if mse > 0 else 100 for mse in MSE]

# Normalize PSNR
norm_PSNR_predicted = preprocessing.normalize([PSNR]).flatten()
PSNR_max, PSNR_min = max(norm_PSNR_predicted), min(norm_PSNR_predicted)

# Compute Frame Score
Predicted_test = [(x - PSNR_min) / (PSNR_max - PSNR_min) for x in norm_PSNR_predicted]

# Save Frame Scores
df_test = pd.DataFrame({"Frame Score": Predicted_test})
df_test.to_excel("frame_scores_test.xlsx", index=False)

# Load threshold value
file_path = "frame_scores_train.xlsx"
df_threshold = pd.read_excel(file_path)

Fs_th = df_threshold["Threshold"].iloc[0]

# Apply Anomaly Detection
df_test["Anomaly_Status"] = df_test["Frame Score"].apply(lambda x: "Normal" if x > Fs_th else "Anomalous")
df_test.to_excel("frame_scores_test.xlsx", index=False)


